In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp
import time as tm
import pysindy as ps
import pandas as pd
import io
import sys

In [2]:
def alphalap(u, alpha):
    utilde=np.fft.fftn(u)
    alphalap=np.real(np.fft.ifftn(-k2**(alpha/2)*utilde))
    return alphalap

def fractionaldiffusion(t,u, alpha):
    lap=alphalap(u.reshape((n,n)),alpha)
    return lap.ravel()

In [3]:
n=128
L=10
T=1.0
nt=201
t = np.linspace(0, T, nt)
dt = T / (nt - 1)
alpha = 1
num_train_traj = 100
num_test_traj = 10
subdomain = [2, 2]

#define these first so they don't have to be reallocated each step
k=np.zeros((n,n,2)) 
k[:n//2+1,:,0]=2*np.pi/(L)*np.arange(n//2+1)[:,np.newaxis]
k[n//2+1:,:,0]=2*np.pi/(L)*(1-n//2+np.arange(n//2-1)[:,np.newaxis])
k[:,:n//2+1,1]=2*np.pi/(L)*np.arange(n//2+1)[np.newaxis,:]
k[:,n//2+1:,1]=2*np.pi/(L)*(1-n//2+np.arange(n//2-1)[np.newaxis,:])
k2=(k[:,:,0]**2+k[:,:,1]**2)

# construct S-grid
spatial_grid = np.zeros((n, n, 2))
spatial_grid[:, :, 0] = np.arange(n)[:, np.newaxis] * L / n
spatial_grid[:, :, 1] = np.arange(n)[np.newaxis, :] * L / n

# Construct ST-grid
x = np.arange(n)*L/n
y = np.arange(n)*L/n
X, Y, time = np.meshgrid(x, y, t)
XY = np.asarray([X, Y, time])
spatiotemporal_grid = np.transpose(XY, axes=[2, 1, 3, 0])

print("ST shape:", np.shape(spatiotemporal_grid))

ST shape: (128, 128, 201, 3)


In [4]:
# # Generate random train trajectories
# train_trajs = []
# IC = []
# for i in range(num_train_traj):
#     print(i, "/", num_train_traj)
#     # define IC
#     const = np.random.randint(10)
#     v1 = np.random.randint(10)
#     v2 = np.random.randint(10)
#     v3 = np.random.randint(10)
    
#     u0=const+(np.sin(v1*np.pi*np.arange(n)[:,np.newaxis]/n)+np.cos(v2*np.pi*np.arange(n)[np.newaxis,:]/n)+np.cos(v3*np.pi*(np.arange(n)[np.newaxis,:]/n+np.arange(n)[:,np.newaxis]/n)))
#     IC.append(u0)
    
#     s = tm.time()
#     usol0 = solve_ivp(
#         fractionaldiffusion, (0, T), y0=u0.ravel(), t_eval=t, 
#         args=(alpha,), method='RK45', rtol=1e-6, atol=1e-6,
#     )
#     print(tm.time()-s)
    
#     train_trajs.append(usol0.y.reshape(n, n, nt, 1))

# np.save("train_traj", train_trajs)

train_trajs = np.load("train_traj.npy")

In [5]:
# # Generate random test trajectories
# test_trajs = []
# IC = []
# for i in range(num_test_traj):
#     print(i, "/", num_test_traj)
#     # define IC
#     const = np.random.randint(10)
#     v1 = np.random.randint(10)
#     v2 = np.random.randint(10)
#     v3 = np.random.randint(10)
    
#     u0=const+(np.sin(v1*np.pi*np.arange(n)[:,np.newaxis]/n)+np.cos(v2*np.pi*np.arange(n)[np.newaxis,:]/n)+np.cos(v3*np.pi*(np.arange(n)[np.newaxis,:]/n+np.arange(n)[:,np.newaxis]/n)))
#     IC.append(u0)
    
#     s = tm.time()
#     usol0 = solve_ivp(
#         fractionaldiffusion, (0, T), y0=u0.ravel(), t_eval=t, 
#         args=(alpha,), method='RK45', rtol=1e-6, atol=1e-6,
#     )
#     print(tm.time()-s)
    
#     test_trajs.append(usol0.y.reshape(n, n, nt, 1))

# np.save("test_traj", test_trajs)

test_trajs = np.load("test_traj.npy")

In [6]:
# pysindy requires sequence inputs
train_seqs = [train_trajs[i] for i in range(train_trajs.shape[0])]
optimizer = ps.STLSQ(threshold=1e-2, alpha=1e-12, normalize_columns=False)

In [7]:
# import required libs to save results to excel
import openpyxl
from openpyxl import Workbook

# Create an Excel workbook and save the data
wb = Workbook()
ws = wb.active

# Add headers
ws.append(['Training Trajs Used', 'nonloc w/ K = ' + str(subdomain), 'pde'])
nonloc_results = {'avg_nonloc_score': []}
pde_results = {'avg_nonloc_score': []}
training_trajs_used = []

In [8]:

# # loop thru different numbers of trajs to train
# for i in range(6):
#     library_functions = []
#     library_function_names = []
    
#     nonloc_lib = ps.NonlocPDELibrary(
#         library_functions=library_functions,
#         function_names=library_function_names,
#         derivative_order=2, spatiotemporal_grid=spatiotemporal_grid,
#         include_bias=True, differentiation_method=ps.SpectralDerivative, K = subdomain
#     )

#     # make both models
#     model = ps.SINDy(feature_library=nonloc_lib, optimizer=optimizer)

#     # fit both models
#     print("Num of training trajs used:", (i+1)*10)
#     s = tm.time()
#     model.fit(train_seqs[:(i+1)*10], t=dt, multiple_trajectories = True)
#     t = tm.time()
    
#     print("nonloc train time:", t-s)

#     # save thee scores of thee models
#     print("score:")
#     score = 0
#     for j in range(num_test_traj):
#         this_score = model.score(test_trajs[j], t=dt)
#         score += this_score
#     print("avg nonloc score:", score/num_test_traj)
#     print("model:")
#     model.print()
#     nonloc_results['avg_nonloc_score'].append(score/num_test_traj)
#     training_trajs_used.append((i+1)*10)

In [9]:
# for i in range(6):  
#     library_functions = []
#     library_function_names = []
    
#     pde_lib = ps.PDELibrary(
#     library_functions=library_functions, 
#     function_names=library_function_names, 
#     derivative_order=2, spatial_grid=spatial_grid, 
#     include_bias=True, differentiation_method=ps.SpectralDerivative
# )
    
#     model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
    
#     print("Num of training trajs used:", (i+1)*10)
#     s = tm.time()
#     model.fit(train_seqs[:(i+1)*10], t=dt, multiple_trajectories = True)
#     t = tm.time()
#     print("pde train time:", t - s)
#         # save thee scores of thee models
#     print("score:")
#     score = 0
#     for j in range(num_test_traj):
#         this_score = model.score(test_trajs[j], t=dt)
#         score += this_score
#     print("avg pde score:", score/num_test_traj)
#     print("model:")
#     model.print()
#     pde_results['avg_nonloc_score'].append(score/num_test_traj)

In [10]:
# # Add data to the workbook
# for trajs, m1_score, m2_score in zip(training_trajs_used, nonloc_results['avg_nonloc_score'], pde_results['avg_nonloc_score']):
#     ws.append([trajs, m1_score, m2_score])

# # Save the workbook
# wb.save(str(subdomain) + '.xlsx')

In [ ]:
num_train_traj_used = 15
indexing = 0

library_functions = []
library_function_names = []
nonloc_lib = ps.NonlocPDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    derivative_order=2, spatiotemporal_grid=spatiotemporal_grid,
    include_bias=True, differentiation_method=ps.SpectralDerivative, K = [4, 2]
)
# make both models
model = ps.SINDy(feature_library=nonloc_lib, optimizer=optimizer)

# fit both models
print("Num of training trajs used:", num_train_traj_used)
s = tm.time()
model.fit(train_seqs[indexing*num_train_traj_used:(indexing+1)*num_train_traj_used], t=dt, multiple_trajectories = True)
# model.fit(train_seqs[30:90], t=dt, multiple_trajectories = True)
t = tm.time()
print("nonloc train time:", t-s)

# save thee scores of thee models
print("score:")
test_score = 0
for j in range(num_test_traj):
    test_score += model.score(test_trajs[j], t=dt)
print("avg nonloc_test score:", test_score/num_test_traj)

# checking for over/under fitting
train_score = 0
for j in range(num_train_traj_used):
    train_score += model.score(train_trajs[j], t=dt)
print("avg nonloc_train score:", train_score/num_train_traj_used)
print("model:")
model.print()

Num of training trajs used: 15
Predicted Num nonloc feature: 64
